In [2]:
import os
import re
import shutil
import string
import time

In [3]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses


In [4]:
from tensorflow.keras import Sequential

In [5]:
import tensorflow_hub as hub

In [6]:
import matplotlib.pyplot as plt

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.12.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.8 MB/s eta 0:00:00


In [ ]:
!pip install -q tf-models-official==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 23.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 90.8 MB/s eta 0:00:00


In [ ]:
import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [7]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
#dataset_dir ="C:\\TEMP\\IMDB\\aclImdb"

84125825/84125825 [==============================] - 9s 0us/step


In [8]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)
print(train_dir)

./aclImdb/train


In [9]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [10]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [11]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [12]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [13]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [14]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [15]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [16]:
embeddingDimensions = 16
vocablaryLength=len(vectorize_layer.get_vocabulary())

In [17]:
def compileAndFit(train_ds,val_ds, getModel,learning_rate):
    
    results=[]
    for attempt in range(3):
        model=getModel()
        
        s = time.time()
    
        model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))
    
        #model.summary()
    
        history = model.fit(    train_ds,    validation_data=val_ds,    epochs=1000, verbose=1,
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=10,mode='max',restore_best_weights=True)] )

        secs=time.time()-s
        lastEpoch=len(history.history['val_binary_accuracy']) 
        epochs = range(1, lastEpoch + 1)
        
        history_dict = history.history
        acc = history_dict['binary_accuracy']
        val_acc = history_dict['val_binary_accuracy']
 
        # plt.plot(epochs , acc, 'bo', label='Training acc')
        # plt.plot(epochs , val_acc, 'b', label='Validation acc')
        # plt.title('Training and validation accuracy')
        # plt.xlabel('Epochs')
        # plt.ylabel('Accuracy')
        # plt.legend(loc='lower right')

        #plt.show()
 
        loss, accuracy = model.evaluate(val_ds)
        tloss, taccuracy = model.evaluate(train_ds)
        results.append((secs,taccuracy,accuracy,lastEpoch))

    print("\t".join(['Time,s','Training accuracy','Validation accuracy','Epochs']))
    for row in results:
        print("\t".join([str(round(row[0])),str(round(row[1], 3)),str(round(row[2], 3)),str( row[3] )  ]))
    

In [ ]:
# https://www.tensorflow.org/tutorials/keras/text_classification
compileAndFit(train_ds,val_ds,lambda:Sequential([
      layers.Embedding(vocablaryLength, 16),
      layers.Dropout(0.2),
      layers.GlobalAveragePooling1D(),
      layers.Dropout(0.2),
      layers.Dense(1)]),0.001)

Epoch 1/1000
625/625 [==============================] - 56s 88ms/step - loss: 0.6633 - binary_accuracy: 0.6912 - val_loss: 0.6138 - val_binary_accuracy: 0.7736
Epoch 2/1000
625/625 [==============================] - 14s 23ms/step - loss: 0.5478 - binary_accuracy: 0.7999 - val_loss: 0.4976 - val_binary_accuracy: 0.8228
Epoch 3/1000
625/625 [==============================] - 10s 15ms/step - loss: 0.4442 - binary_accuracy: 0.8449 - val_loss: 0.4196 - val_binary_accuracy: 0.8480
Epoch 4/1000
625/625 [==============================] - 6s 10ms/step - loss: 0.3775 - binary_accuracy: 0.8667 - val_loss: 0.3732 - val_binary_accuracy: 0.8618
Epoch 5/1000
625/625 [==============================] - 7s 11ms/step - loss: 0.3346 - binary_accuracy: 0.8790 - val_loss: 0.3448 - val_binary_accuracy: 0.8664
Epoch 6/1000
625/625 [==============================] - 5s 8ms/step - loss: 0.3047 - binary_accuracy: 0.8888 - val_loss: 0.3256 - val_binary_accuracy: 0.8716
Epoch 7/1000
625/625 [======================

In [ ]:
#https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin
#https://www.tensorflow.org/text/guide/word_embeddings
#https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb
compileAndFit(train_ds,val_ds,lambda:Sequential(
    [
        layers.Embedding(vocablaryLength, 16),
        layers.GlobalAveragePooling1D(),
        layers.Dense(16, activation="relu"),
        layers.Dense(1),
    ]),0.001)

Epoch 1/1000
625/625 [==============================] - 40s 63ms/step - loss: 0.5651 - binary_accuracy: 0.7408 - val_loss: 0.3817 - val_binary_accuracy: 0.8506
Epoch 2/1000
625/625 [==============================] - 12s 19ms/step - loss: 0.3055 - binary_accuracy: 0.8822 - val_loss: 0.3026 - val_binary_accuracy: 0.8744
Epoch 3/1000
625/625 [==============================] - 6s 10ms/step - loss: 0.2360 - binary_accuracy: 0.9101 - val_loss: 0.2882 - val_binary_accuracy: 0.8824
Epoch 4/1000
625/625 [==============================] - 5s 9ms/step - loss: 0.1963 - binary_accuracy: 0.9283 - val_loss: 0.2987 - val_binary_accuracy: 0.8772
Epoch 5/1000
625/625 [==============================] - 5s 7ms/step - loss: 0.1656 - binary_accuracy: 0.9417 - val_loss: 0.3030 - val_binary_accuracy: 0.8766
Epoch 6/1000
625/625 [==============================] - 4s 6ms/step - loss: 0.1429 - binary_accuracy: 0.9514 - val_loss: 0.3182 - val_binary_accuracy: 0.8752
Epoch 7/1000
625/625 [=========================

In [ ]:
#https://www.tensorflow.org/tutorials/load_data/text
compileAndFit(train_ds,val_ds,lambda:Sequential([
      layers.Embedding(vocablaryLength, 64),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(1)
  ]),0.001)
#86.4

Epoch 1/1000
625/625 [==============================] - 63s 93ms/step - loss: 0.4900 - binary_accuracy: 0.7510 - val_loss: 0.3505 - val_binary_accuracy: 0.8478
Epoch 2/1000
625/625 [==============================] - 16s 25ms/step - loss: 0.2619 - binary_accuracy: 0.8963 - val_loss: 0.3205 - val_binary_accuracy: 0.8636
Epoch 3/1000
625/625 [==============================] - 8s 12ms/step - loss: 0.1427 - binary_accuracy: 0.9520 - val_loss: 0.3343 - val_binary_accuracy: 0.8664
Epoch 4/1000
625/625 [==============================] - 7s 11ms/step - loss: 0.0592 - binary_accuracy: 0.9869 - val_loss: 0.3802 - val_binary_accuracy: 0.8656
Epoch 5/1000
625/625 [==============================] - 7s 12ms/step - loss: 0.0179 - binary_accuracy: 0.9980 - val_loss: 0.4342 - val_binary_accuracy: 0.8664
Epoch 6/1000
625/625 [==============================] - 4s 6ms/step - loss: 0.0051 - binary_accuracy: 0.9998 - val_loss: 0.4827 - val_binary_accuracy: 0.8666
Epoch 7/1000
625/625 [=======================

In [ ]:
#https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb
compileAndFit(train_ds,val_ds,lambda:Sequential([
   layers.Embedding(vocablaryLength,16),
   layers.Bidirectional( layers.LSTM(64)),
   layers.Dense(64, activation='relu'),
   layers.Dense(1) 
]),0.001)
 




Epoch 1/1000
625/625 [==============================] - 55s 81ms/step - loss: 0.4974 - binary_accuracy: 0.7490 - val_loss: 0.3971 - val_binary_accuracy: 0.8340
Epoch 2/1000
625/625 [==============================] - 22s 35ms/step - loss: 0.3184 - binary_accuracy: 0.8718 - val_loss: 0.3569 - val_binary_accuracy: 0.8616
Epoch 3/1000
625/625 [==============================] - 17s 28ms/step - loss: 0.2295 - binary_accuracy: 0.9151 - val_loss: 0.3624 - val_binary_accuracy: 0.8534
Epoch 4/1000
625/625 [==============================] - 17s 27ms/step - loss: 0.1744 - binary_accuracy: 0.9383 - val_loss: 0.3879 - val_binary_accuracy: 0.8596
Epoch 5/1000
625/625 [==============================] - 14s 22ms/step - loss: 0.1412 - binary_accuracy: 0.9513 - val_loss: 0.4062 - val_binary_accuracy: 0.8598
Epoch 6/1000
625/625 [==============================] - 14s 23ms/step - loss: 0.1178 - binary_accuracy: 0.9592 - val_loss: 0.4908 - val_binary_accuracy: 0.8518
Epoch 7/1000
625/625 [==================

In [ ]:
#same as above but different lr
#https://www.tensorflow.org/text/tutorials/text_classification_rnn
compileAndFit(train_ds,val_ds,lambda:Sequential([
     layers.Embedding(vocablaryLength, 64, mask_zero=True),
     layers.Bidirectional( layers.LSTM(64)),
     layers.Dense(64, activation='relu'),
     layers.Dense(1)
]),0.0001)
#86.8

Epoch 1/1000
625/625 [==============================] - 68s 90ms/step - loss: 0.5839 - binary_accuracy: 0.6808 - val_loss: 0.4237 - val_binary_accuracy: 0.8328
Epoch 2/1000
625/625 [==============================] - 25s 40ms/step - loss: 0.3259 - binary_accuracy: 0.8716 - val_loss: 0.3301 - val_binary_accuracy: 0.8644
Epoch 3/1000
625/625 [==============================] - 18s 29ms/step - loss: 0.2379 - binary_accuracy: 0.9135 - val_loss: 0.3405 - val_binary_accuracy: 0.8568
Epoch 4/1000
625/625 [==============================] - 17s 28ms/step - loss: 0.1904 - binary_accuracy: 0.9334 - val_loss: 0.3355 - val_binary_accuracy: 0.8616
Epoch 5/1000
625/625 [==============================] - 17s 27ms/step - loss: 0.1537 - binary_accuracy: 0.9488 - val_loss: 0.3491 - val_binary_accuracy: 0.8696
Epoch 6/1000
625/625 [==============================] - 16s 25ms/step - loss: 0.1309 - binary_accuracy: 0.9572 - val_loss: 0.3952 - val_binary_accuracy: 0.8660
Epoch 7/1000
625/625 [==================

In [ ]:
#https://www.tensorflow.org/text/tutorials/text_classification_rnn
compileAndFit(train_ds,val_ds,lambda:Sequential([
         layers.Embedding( vocablaryLength, 64, mask_zero=True),
         layers.Bidirectional(layers.LSTM(64,  return_sequences=True)),
         layers.Bidirectional(layers.LSTM(32)),
         layers.Dense(64, activation='relu'),
         layers.Dropout(0.5),
         layers.Dense(1)
        ]),0.0001)


Epoch 1/1000
625/625 [==============================] - 89s 116ms/step - loss: 0.5769 - binary_accuracy: 0.6733 - val_loss: 0.3652 - val_binary_accuracy: 0.8460
Epoch 2/1000
625/625 [==============================] - 36s 57ms/step - loss: 0.2987 - binary_accuracy: 0.8824 - val_loss: 0.3194 - val_binary_accuracy: 0.8588
Epoch 3/1000
625/625 [==============================] - 32s 51ms/step - loss: 0.2216 - binary_accuracy: 0.9202 - val_loss: 0.3379 - val_binary_accuracy: 0.8730
Epoch 4/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.1702 - binary_accuracy: 0.9430 - val_loss: 0.3496 - val_binary_accuracy: 0.8578
Epoch 5/1000
625/625 [==============================] - 29s 47ms/step - loss: 0.1345 - binary_accuracy: 0.9571 - val_loss: 0.3685 - val_binary_accuracy: 0.8728
Epoch 6/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.1088 - binary_accuracy: 0.9675 - val_loss: 0.4560 - val_binary_accuracy: 0.8714
Epoch 7/1000
625/625 [=================

In [18]:
#https://www.tensorflow.org/tutorials/keras/text_classification_with_hub
#https://www.tensorflow.org/hub/tutorials/tf2_text_classification
compileAndFit(raw_train_ds,raw_val_ds,lambda:Sequential([
         hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2", input_shape=[],dtype=tf.string, trainable=True),
         layers.Dense(16, activation='relu'),
         layers.Dense(1)
        ]),0.001)


Epoch 1/1000
625/625 [==============================] - 112s 169ms/step - loss: 0.4285 - binary_accuracy: 0.8106 - val_loss: 0.3138 - val_binary_accuracy: 0.8676
Epoch 2/1000
625/625 [==============================] - 60s 95ms/step - loss: 0.1801 - binary_accuracy: 0.9342 - val_loss: 0.3073 - val_binary_accuracy: 0.8758
Epoch 3/1000
625/625 [==============================] - 37s 59ms/step - loss: 0.0800 - binary_accuracy: 0.9765 - val_loss: 0.3817 - val_binary_accuracy: 0.8688
Epoch 4/1000
625/625 [==============================] - 26s 42ms/step - loss: 0.0267 - binary_accuracy: 0.9944 - val_loss: 0.4648 - val_binary_accuracy: 0.8644
Epoch 5/1000
625/625 [==============================] - 20s 33ms/step - loss: 0.0077 - binary_accuracy: 0.9995 - val_loss: 0.5148 - val_binary_accuracy: 0.8684
Epoch 6/1000
625/625 [==============================] - 20s 31ms/step - loss: 0.0031 - binary_accuracy: 0.9998 - val_loss: 0.5618 - val_binary_accuracy: 0.8684
Epoch 7/1000
625/625 [================

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/google/electra_base/2'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)
  

In [ ]:
compileAndFit(raw_train_ds,raw_val_ds,build_classifier_model,0.00003)

Epoch 1/1000
625/625 [==============================] - 193s 230ms/step - loss: 0.2868 - binary_accuracy: 0.8795 - val_loss: 0.2249 - val_binary_accuracy: 0.9084
Epoch 2/1000
625/625 [==============================] - 111s 177ms/step - loss: 0.1684 - binary_accuracy: 0.9345 - val_loss: 0.2785 - val_binary_accuracy: 0.8988
Epoch 3/1000
625/625 [==============================] - 108s 172ms/step - loss: 0.0946 - binary_accuracy: 0.9666 - val_loss: 0.4400 - val_binary_accuracy: 0.8862
Epoch 4/1000
625/625 [==============================] - 106s 170ms/step - loss: 0.0608 - binary_accuracy: 0.9791 - val_loss: 0.4326 - val_binary_accuracy: 0.8998
Epoch 5/1000
625/625 [==============================] - 106s 170ms/step - loss: 0.0426 - binary_accuracy: 0.9859 - val_loss: 0.4546 - val_binary_accuracy: 0.8954
Epoch 6/1000
625/625 [==============================] - 108s 172ms/step - loss: 0.0289 - binary_accuracy: 0.9900 - val_loss: 0.4276 - val_binary_accuracy: 0.9032
Epoch 7/1000
625/625 [======